# Read Libs

In [10]:
import pandas as pd
import datetime as dt
import numpy as np
import requests as req
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementClickInterceptedException

import time



# display and output settings
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, HTML



display(HTML('''
<style>
    .container { width:85% !important; }
</style>
'''))

pd.set_option('display.min_rows', 100)

# Read in Tournament Details

In [11]:
path_tournament_detail = "../data/transformed_data/tournament_info/2024-10-06 16h20m45s/tournament_details_transformed.csv"
df_tournament_details = pd.read_csv(path_tournament_detail)

## Get Cities Dataframe

In [12]:
# Extract unique cities from the df_tournament_details dataframe
unique_cities = df_tournament_details['city'].dropna().unique()

# Create a new dataframe with the unique cities
df_unique_cities = pd.DataFrame(unique_cities, columns=['City'])

#initialize a new column to store the base url
df_unique_cities['base_url'] = ""

In [13]:
# Create a flag column to indicate if the city name contains "Club", "GC", or "CC"
"""
This script processes a DataFrame `df_unique_cities` to create a new column `is_golf_club`.
The new column is a flag that indicates whether the city name contains the words "Club", "GC", or "CC".

- `(Club|GC|CC)` is a group that matches any of the words "Club", "GC", or "CC".

The script then displays the updated DataFrame `df_unique_cities`.
"""

df_unique_cities['is_golf_club'] = df_unique_cities['City'].str.contains(r'(Club|GC|CC|Course|Golf)', regex=True)


'\nThis script processes a DataFrame `df_unique_cities` to create a new column `is_golf_club`.\nThe new column is a flag that indicates whether the city name contains the words "Club", "GC", or "CC".\n\n- `(Club|GC|CC)` is a group that matches any of the words "Club", "GC", or "CC".\n\nThe script then displays the updated DataFrame `df_unique_cities`.\n'

/var/folders/sj/kb7kr7w572z54pgv0pftb9v00000gn/T/ipykernel_50514/3291710705.py:11: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_unique_cities['is_golf_club'] = df_unique_cities['City'].str.contains(r'(Club|GC|CC|Course|Golf)', regex=True)


In [14]:
only_cities_temp = df_unique_cities[df_unique_cities['is_golf_club'] == False]

print('difference when adding golf course flag')
df_unique_cities['City'].count()
only_cities_temp['City'].count()

difference when adding golf course flag


413

273

# Get Base URLs for each City

### Start chromedriver

In [15]:
# Specify the path to the ChromeDriver executable
chrome_driver_path = '../drivers/chromedriver_arm'
# firefox_driver_path = '../drivers/geckodriver'

#link for solution to cannot open chromedriver
# https://stackoverflow.com/questions/60362018/macos-catalinav-10-15-3-error-chromedriver-cannot-be-opened-because-the-de
# xattr -d com.apple.quarantine


driver = webdriver.Chrome(executable_path=chrome_driver_path)


/var/folders/sj/kb7kr7w572z54pgv0pftb9v00000gn/T/ipykernel_50514/2826122168.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_driver_path)


In [16]:
counter = 0
for city_name in df_unique_cities[df_unique_cities['is_golf_club'] == False]['City']:

    # Open the Weather Underground history page

    #setting timeout for page load
    driver.set_page_load_timeout(5)

    #fetch base url
    driver.get("https://www.wunderground.com/history")

    # Wait for the page to load and locate the city input field
    city_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "historySearch"))
    )

    try:
    # Enter the city name
        city_input.clear()
        city_input.send_keys(city_name)
        time.sleep(2)
        city_input.send_keys(Keys.RETURN)
    
    # If the element is not found, try again
    except StaleElementReferenceException:
        city_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "historySearch"))
        )
        city_input.clear()
        city_input.send_keys(city_name)
        time.sleep(2)
        city_input.send_keys(Keys.RETURN)

    #setting timeout for page load for form submission to 3
    # driver.set_page_load_timeout(3)

    # Submit the form
    submit_button_xpath = "/html/body/app-root/app-history-search/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div[2]/section/div[2]/div/div/div/div/form/lib-date-selector/div/input"
    submit_button = driver.find_element(By.XPATH, submit_button_xpath)

    #submitting button with javacript if the button is not clickable with interception of element
    try:
        submit_button.click()    
    except ElementClickInterceptedException:
        print('element intercepted submit javascript') 
        driver.execute_script("arguments[0].click();", submit_button)
    

    # Wait for the page to load and locate the title element
    title_xpath = "/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div[2]/section/div[1]/lib-city-header/div[1]/div/h1/span[1]"

    # Wait for the page to load and locate the title element
    try:
        title_element = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, title_xpath))
        )
        current_url = driver.current_url
        load_status= "Page loaded successfully"


    # Page takes too long to load so stop it and store the current URL
    # solution for this
    # https://stackoverflow.com/questions/36299228/selenium-webdriver-takes-too-long-to-load-a-page
    except TimeoutException:
        driver.execute_script("window.stop();")
        current_url = driver.current_url
        load_status = "Page Timed out"

    try:
        # Update the base_url for the given city
        df_unique_cities.loc[df_unique_cities['City'] == city_name, 'base_url'] = current_url
    except:
        df_unique_cities.loc[df_unique_cities['City'] == city_name, 'base_url'] = "Not Found"
    
    
    print(f"{counter} : City: {city_name}: url: {current_url}: Load Status: {load_status}")
    counter += 1



0 : City:  Kapalua, HI: url: https://www.wunderground.com/history/daily/RJOA/date/2024-10-26: Load Status: Page loaded successfully
1 : City:  Tucson, AZ: url: https://www.wunderground.com/history/daily/KTUS/date/2024-10-26: Load Status: Page Timed out
2 : City:  Honolulu, HI: url: https://www.wunderground.com/history/daily/PHNL/date/2024-10-26: Load Status: Page loaded successfully
3 : City:  Scottsdale, AZ: url: https://www.wunderground.com/history/daily/KPHX/date/2024-10-26: Load Status: Page loaded successfully
4 : City:  Pacific Palisades, CA: url: https://www.wunderground.com/history/daily/KLAX/date/2024-10-26: Load Status: Page loaded successfully
5 : City:  Doral, FL: url: https://www.wunderground.com/history/daily/KMIA/date/2024-10-26: Load Status: Page loaded successfully
6 : City:  Orlando, FL: url: https://www.wunderground.com/history: Load Status: Page Timed out
7 : City:  Ponte Vedra Beach, FL: url: https://www.wunderground.com/history/daily/KSGJ/date/2024-10-26: Load Sta

## Save Base URL's to File

In [17]:
output_path = "../data/raw_data/weather/weather_city_base_urls/"
timestamp_run = dt.datetime.now().strftime('%Y-%m-%d %Hh%Mm%Ss')

os.mkdir(os.path.join(output_path, timestamp_run))

df_unique_cities.to_csv(f'{output_path}/{timestamp_run}/weather_city_base_urls.csv', index=False)